In [1]:
import os
import glob
import monai
import torch
import shutil
import argparse
import warnings
import datetime
import numpy as np
from torch import nn
import nibabel as nib
from pathlib import Path
from unet_3d import Unet_3d
import monai.transforms as mt
import pytorch_lightning as pl
from torchmetrics import IoU, F1
from attn_unet_3d import Attn_UNet3d
from matplotlib import pyplot as plt
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from torchmetrics.functional import dice_score, iou
from data_3d import train_loader_ACDC, val_loader_ACDC
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator as ea

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
torch.manual_seed(12345)

ModuleNotFoundError: No module named 'unet_3d'

In [8]:
data_dir = 'dataset'
images = [os.path.join(data_dir, 'images', f) for f in os.listdir(os.path.join(data_dir, 'images'))]
labels = [os.path.join(data_dir, 'labels', f) for f in os.listdir(os.path.join(data_dir, 'labels'))]

# # Ensure each image has a corresponding label
# assert len(images) == len(labels)

# Prepare the data dictionary
data_dicts = [{'image': img, 'label': lbl} for img, lbl in zip(images, labels)]

# Split the dataset (80% train, 20% validation)
train_files, val_files = data_dicts[:int(len(data_dicts)*0.8)], data_dicts[int(len(data_dicts)*0.8):]


In [11]:
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    # AddChanneld(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    RandRotate90d(keys=['image', 'label'], prob=0.5),
    ToTensord(keys=['image', 'label']),
])

val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    # AddChanneld(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    ToTensord(keys=['image', 'label']),
])


In [12]:
# Create MONAI datasets
train_ds = Dataset(data=train_files, transform=train_transforms)
val_ds = Dataset(data=val_files, transform=val_transforms)

# Create PyTorch Data Loaders
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)


In [20]:
from monai.networks.nets import UNet

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2)
)

loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4)

In [24]:
num_epochs = 5
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
dice_metric = DiceMetric(include_background=True, reduction="mean")

for epoch in range(num_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{num_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    # Validation
    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            for val_data in val_loader:
                val_inputs, val_labels = val_data["image"]

----------
epoch 1/5


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\nuzul\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\transforms\transform.py", line 141, in apply_transform
    return _apply_transform(transform, data, unpack_items, lazy, overrides, log_stats)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nuzul\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\transforms\transform.py", line 98, in _apply_transform
    return transform(data, lazy=lazy) if isinstance(transform, LazyTrait) else transform(data)
                                                                               ^^^^^^^^^^^^^^^
  File "c:\Users\nuzul\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\transforms\io\dictionary.py", line 162, in __call__
    data = self._loader(d[key], reader)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nuzul\